# Setup and Imports

In [1]:
%%capture
!rm -rf fairgraph
!git clone https://github.com/GMattheisen/fairgraph.git
!pip install -r ./fairgraph/requirements.txt
!pip install -U ./fairgraph

In [3]:
from fairgraph import KGClient
import os
import re
import io
import logging
from datetime import date, datetime
from pprint import pprint
import yaml
from collections import defaultdict
import requests
import numpy as np
from neo.io import get_io, Spike2IO, NeoMatlabIO, RawBinarySignalIO # dependent on data format
from fairgraph import KGClient
from fairgraph.commons import (Species, Handedness, Sex, Strain, Age, Genotype, QuantitativeValue, 
                                BrainRegion, CellType, QuantitativeValueRange, StimulusType)
from fairgraph.core import (Person, Identifier, Material, Organization, Address, Subject, Protocol, Step, use_namespace)
from fairgraph.experiment import (Device, ElectrophysiologicalStimulation,  
                                ElectrophysiologicalStimulus, BehavioralStimulation, BehavioralStimulus, CranialWindow, Craniotomy)
from fairgraph.electrophysiology import (Distribution, PatchClampActivity, PatchClampExperiment,
                                         Trace, MultiChannelMultiTrialRecording, Distribution, QualifiedMultiTraceGeneration,
                                        ElectrodePlacementActivity, ElectrodeImplantationActivity, Sensor, ImplantedBrainTissue,
                                        ECoGExperiment, EEGExperiment)
from fairgraph.base import KGQuery, as_list
from fairgraph.minds import Dataset

use_namespace('neuralactivity')

/Users/glynismattheisen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
logging.basicConfig(filename="to_knowledge_graph.log",
                    filemode='a',
                    level=logging.DEBUG)

logger = logging.getLogger("nar")

token = ""
client = KGClient(token, nexus_endpoint='https://nexus-int.humanbrainproject.org/v0')

if 'int' in client.nexus_endpoint:
    Subject._path = '/core/subject/v0.1.2'
    Craniotomy._path = '/optophysiology/craniotomy/v0.4.0'
    CranialWindow._path = '/optophysiology/cranialwindow/v0.1.1'
    Device._path = '/electrophysiology/device/v0.1.2'
    Sensor._path = '/electrophysiology/sensor/v0.1.0'
    Material._path = '/core/material/v0.1.2'
    Step._path = '/core/protocol/v0.1.6'
    Protocol._path = '/core/protocol/v0.1.6'
    BehavioralStimulation._path = '/experiment/behavioralstimulation/v0.1.0'
    BehavioralStimulus._path = '/experiment/behavioralstimulus/v0.1.0'
    ElectrophysiologicalStimulation._path = '/optophysiology/electrophysiologicalstimulation/v0.5.0'
    ElectrophysiologicalStimulus._path = '/optophysiology/electrophysiologicalstimulus/v0.1.0'    
    ImplantedBrainTissue._path = '/core/implantedbraintissue/v0.1.0'
    ECoGExperiment._path = '/electrophysiology/electrodearrayexperiment/v0.1.1'
    ElectrodeImplantationActivity._path = '/experiment/electrodeplacement/v0.1.0'
    QualifiedMultiTraceGeneration._path = '/electrophysiology/multitracegeneration/v0.2.3'
    MultiChannelMultiTrialRecording._path = '/electrophysiology/multitrace/v0.3.1'

### ECoG

#### Components

Subject<br>
Craniotomy<br>
CranialWindow<br>
Device<br>
Sensor<br>
Step<br>
Protocol<br>
Stimulus<br>
Stimulation<br>
ImplantedBrainTissue<br>
ECoGExperiment<br>
ElectrodeImplantationActivity<br>
QualifiedMultiTraceGeneration<br>
MultiChannelMultiTrialRecording

# Locate Dataset and Define Contributors

In [5]:
dataset_name = None
minds_dataset = Dataset.by_name(dataset_name, client, resolved=True, api="query")

# map MINDSPerson to Person (for owner, contributors)
owner = [obj.resolve(client) for obj in as_list(minds_dataset.owners)]
contributors = [obj.resolve(client) for obj in as_list(minds_dataset.contributors)]

In [ ]:
for minds_person in contributors:
    print(minds_person)

In [7]:
"""People"""
abc = Person(given_name="", family_name="", email="")
abc.save(client)
deg = Person(given_name="", family_name="", email=None)
deg.save(client)
hij = Person(given_name="", family_name="", email=None)
hij.save(client)

people = [abc, deg, hij]

specimen_group = [obj.resolve(client) for obj in as_list(minds_dataset.specimen_group)]
minds_subjects = []
for sg in specimen_group:
    minds_subjects.extend([obj.resolve(client) for obj in as_list(sg.subjects)])

[Person(identifier='3e1125b33c46e59da87609e3af86545d', name='Sanchez-Vives, Maria', shortname='Sanchez-Vives, M.', id=https://nexus.humanbrainproject.org/v0/data/minds/core/person/v1.0.0/0da33b37-3118-4ff3-b278-a9a12034adaf)]
Person(identifier='3e1125b33c46e59da87609e3af86545d', name='Sanchez-Vives, Maria', shortname='Sanchez-Vives, M.', id=https://nexus.humanbrainproject.org/v0/data/minds/core/person/v1.0.0/0da33b37-3118-4ff3-b278-a9a12034adaf)


# Define Subjects, Samples, Experiment

In [ ]:
for minds_subject in minds_subjects:
    print(minds_subject.name)
    print(minds_subject.samples)
    for minds_sample in as_list(minds_subject.samples):
        print(as_list(minds_subject.samples))
        minds_sample = minds_sample.resolve(client)
        print(minds_sample.name)

In [ ]:
"""Import trace information from previous run if available"""

if os.path.exists(f'{minds_dataset.name}_trace_dict.yaml'):
    print("loading trace_dict")
    with open(f'{minds_dataset.name}_trace_dict.yaml') as file:
        trace_dict = yaml.load(file, Loader=yaml.FullLoader)
        preload = True
        print(trace_dict)
else:
    print("no saved trace_dict, building trace_dict")
    trace_dict = {}
    preload = False

In [17]:
# map MINDSSubject to Subject
for minds_subject in minds_subjects:
    print(minds_subject.name)

    """Subject"""
    species = minds_subject.species.resolve(client).name
    strain = minds_subject.strains
    genotype = minds_subject.genotype
    sex = minds_subject.sex.resolve(client).name
    if sex == "Unknown" or sex == '<Unknown>':
        sex = None
    else:
        sex=Sex(sex)
    handedness = None
    age_units = "days"
    age_value = minds_subject.age
    if "-" in age_value:
        min, max = age_value.split("-")
        age_value = QuantitativeValueRange(int(min), int(max), age_units)
    else:
        age_value = QuantitativeValue(int(age_value), age_units)    
    age_category = "Post-natal"
    death_date = None # date type
    group = None # 'control group' v 'treatment group'

    subject = Subject(
      minds_subject.name,
      species=Species(species),
      strain=Strain(strain),
      genotype=Genotype(genotype),
      sex = sex,
      age=Age(age_value, age_category),
      death_date=None
      )
    print(subject)
    subject.save(client)
    assert subject.id is not None
        
    for minds_sample in as_list(minds_subject.samples):
        minds_sample = minds_sample.resolve(client)
        
        brain_regions = [BrainRegion("striatum")]  
        
        cranial_window = CranialWindow(
            name = f"Cranial window for {subject.name}",
            brain_location = brain_location,
            window_type = "",
            diameter = QuantitativeValue(int(value), "units"),
            fluorescence_labeling = "",
            description = ""
        )
        cranial_window.save(client)
    
        craniotomy = Craniotomy(
            name = f"Craniotomy for {subject.name}",
            subject = subject,
            cranial_window = cranial_window,
            anesthesia = "",      
            protocol = Protocol(),
            start_time =, # datetime
            end_time = , # datetime
            people = people
        )
        craniotomy.save(client)

        device = Device(
            name = "Multiclamp 700B amplifier",
            manufacturer="Molecular Devices",
            model_name="700B",
            software_version =None,
            serial_number = None,
            distribution = Distribution("https://www.moleculardevices.com/sites/default/files/en/assets/data-sheets/dd/cns/multiclamp-700b-microelectrode-amplifier.pdf"),
            )
        device.save(client)
        print(device)
        
        sensors = Sensor(
            name = "Sensors",
            coordinate_system = Distribution(""),
            coordinate_units = "",
            description = ""
        )
        sensors.save(client)

        step_one = Step(
            name = "Step one for protocol",
            previous_step_name = None,
            sequence_number = 2,
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            description = "",
            materials - Material(), # or list
            author = Person() # or list
            )
        step_one.save(client)
            
        step_two = Step(
            name = "Step two for protocol",
            previous_step_name = "Step one for protocl",
            sequence_number = 2,
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            description = "",
            materials - Material(), # or list
            author = Person() # or list
            )
        step_two.save(client)

        steps = [step_one, step_two]
        
        protocol = Protocol(
            name = "Protcol for ",
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            number_of_steps = 0,
            steps = steps, 
            materials - Material(), # or list
            author = Person(), # or list
            date_published = # date type
            )
        protocol.save(client)
    
        electrophysiological_stimulus = ElectrophysiologicalStimulus(
            name=f"Stimulus for {minds_sample.name}",
            description = "",
            distribution = None
            )
        electrophysiological_stimulus.save(client)

        electrophysiological_stimulation = ElectrophysiologicalStimulation(
            name=f"Stimulation for {minds_sample.name}",
            stimulus = electrophysiological_stimulus,
            protocol= protocol,
            citation = None,
            code = None,
            license = None
            )
        electrophysiological_stimulation.save(client)
        print(electrophysiological_stimulation)
            
        expt = ECoGExperiment(
            name = f"ECoG recording for {minds_sample.name}",
            device=device,
            stimulation = [electrophysiological_stimulation],
            sensors=sensors,
            digitized_head_points_coordinates = Sensor(),
            head_localization_coils_coordinates = Sensor(),
            digitized_head_points = , # bool
            digitized_landmarks = , # bool
            start_time =, # datetime
            end_time = , # datetime
            people = people,
            protocol = Protocol()
        )
        expt.save(client)
        
        implanted_brain_tissue = ImplantedBrainTissue(
            name=f'Tissue from {minds_sample.name}',
            subject=subject
        )
        implanted_brain_tissue.save(client)
        
        electrode = ElectrodeImplantationActivity(
            name = f"Electrode implantation activity for {minds_sample.name}",
            subject = subject,
            brain_location = brain_regions,
            implanted_brain_tissues = implanted_brain_tissue,
            device=device,
            protocol = Protocol(),
            cranial_window = cranial_window,
            anesthesia = "",
            start_time =, # datetime
            end_time = , # datetime
            people=people)
        electrode.save(client)
        
        
        file_dir = "hbp-d00000/"

        for item in os.listdir(file_dir):
            filename=item
            file_url=file_dir + filename
            print("file_url", file_url)
            file_location = "https://object.cscs.ch/v1/AUTH_0000000000000000000/" + file_url
            print("file_location", file_location)
        
            io = RawBinarySignalIO(filename = file_url)
            data = io.read_block()
            n_segments = len(data.segments)
            sigs = data.segments[0].analogsignals
            channel_names = [sig.name for sig in sigs]
            units = [sig.units.dimensionality.string for sig in sigs]
            time_step = QuantitativeValue(**{
                                "unit_text": "ms",
                                "value": float(sigs[0].sampling_period.rescale("ms"))
                              })
            
        """Trace Generation"""
        """Accounts for muliple files existing per sample"""
        
        holding_potential_value = None
        holding_potential_unit = None
        
        if preload: # building from YAML
            print("building trace metadata from YAML")
            trace_generation = QualifiedMultiTraceGeneration(
                f"Metadata for ECoG recording in {minds_sample.name}",
                stimulus_experiment=expt,
                sweeps=trace_dict[minds_sample.name]['n_segments'],
                channel_type = None,
                holding_potential = QuantitativeValue(holding_potential_value, holding_potential_unit),
                sampling_frequency = None, 
                power_line_frequency = None
                )
            trace_generation.save(client)
            print(trace_generation)

            trace = MultiChannelMultiTrialRecording(
                f"Traces recorded in {minds_sample.name}",
                data_location=Distribution(location = trace_dict[minds_sample.name]['file_location'],
                  content_type=trace_dict[minds_sample.name]['content_type']),
                generated_by=expt,
                generation_metadata=trace_generation,
                channel_names=trace_dict[minds_sample.name]['channel_names'],
                data_unit=trace_dict[minds_sample.name]['data_unit'],
                time_step=QuantitativeValue(trace_dict[minds_sample.name]['time_step'], trace_dict[minds_sample.name]['time_step_unit']),
                channel_type = None,
                part_of=minds_dataset
                )
            print(trace)
            trace.save(client)
        
        else:
            file_dir = "hbp-d00000/"
            filename = os.listdir(file_dir)
            file_url=file_dir + filename
            file_location = "https://object.cscs.ch/v1/AUTH_0000000000000000000/" + file_url

            io = RawBinarySignalIO(filename = file_url)
            data = io.read_block()
            n_segments = len(data.segments)
            sigs = data.segments[0].analogsignals
            channel_names = [sig.name for sig in sigs]
            units = [sig.units.dimensionality.string for sig in sigs]
            time_step_unit = "ms"
            time_step = float(sigs[0].sampling_period.rescale(time_step_unit))
            content_type = ""
            
            print("trace_dict does not exist, building trace dict") #
            trace_dict[minds_sample.name] = {}
            trace_dict[minds_sample.name]['channel_names'] = channel_names
            trace_dict[minds_sample.name]['data_unit'] = units
            trace_dict[minds_sample.name]['time_step'] = time_step
            trace_dict[minds_sample.name]['time_step_unit'] = time_step_unit
            trace_dict[minds_sample.name]['n_segments'] = n_segments
            trace_dict[minds_sample.name]['file_url'] = file_url
            trace_dict[minds_sample.name]['file_location'] = file_location
            trace_dict[minds_sample.name]['content_type'] = content_type

            trace_generation = QualifiedMultiTraceGeneration(
                f"Metadata for ECoG recording in {minds_sample.name}",
                stimulus_experiment = expt,
                sweeps = n_segments,
                channel_type = None,
                holding_potential = QuantitativeValue(holding_potential_value, holding_potential_unit),
                sampling_frequency = None, 
                power_line_frequency = None
                )
            trace_generation.save(client)
            print(trace_generation)

            trace = MultiChannelMultiTrialRecording(
                f"Traces recorded in {minds_sample.name}",
                data_location=Distribution(location = file_location,
                  content_type=content_type),
                generated_by=expt,
                generation_metadata=trace_generation,
                channel_names=channel_names,
                data_unit=units,
                time_step=QuantitativeValue(**{
                "unit_text": time_step_unit,
                "value": time_step
                }),
                channel_type = None,
                part_of=minds_dataset
                )
            print(trace)
            trace.save(client)

Subject(name='FXS_KO_S1', species=Species('Mus musculus', 'http://purl.obolibrary.org/obo/NCBITaxon_10090'), strain=Strain('ATJ/FVB.129P2-FMR1-mix', 'https://dknet.org/data/record/nlx_154697-1/IMSR_JAX:004624/resolver?q=%2A&l=&filter[]=Catalog%20Number:004624&i=5d422ff90fbc3b6016f6f089'), sex=Sex('male', 'schema:Male'), age=Age(QuantitativeValue(3 'months'), 'Post-natal'), id=None)
SUBJECT FXS_KO_S1
SAMPLE FXS_KO_S1_samp-1
ECoGExperiment(name='ECoG recording for FXS_KO_S1_samp-1', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), stimulus=[BehavioralStimulation(name='Stimulation for FXS_KO_S1', behavioral_stimulus=BehavioralStimulus(name='Stimulus for FXS_KO_S1', description='Evoked with vibrator in whiskers; chan

MINDS SAMPLE FXS_KO_S1_samp-3
DICT hbp-01853_cond-FXS_type-KO_sub-1_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
FILE NAME hbp-01853_cond-FXS_type-KO_sub-1_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
FILE URL hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-1/hbp-01853_cond-FXS_type-KO_sub-1_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-1/hbp-01853_cond-FXS_type-KO_sub-1_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
SUBJECT FXS_KO_S1
SAMPLE FXS_KO_S1_samp-4
ECoGExperiment(name='ECoG recording for FXS_KO_S1_samp-4', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), st

MINDS SAMPLE FXS_KO_S2_samp-1
DICT hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
FILE NAME hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
FILE URL hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-2/hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-2/hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
SUBJECT FXS_KO_S2
SAMPLE FXS_KO_S2_samp-2
ECoGExperiment(name='ECoG recording for FXS_KO_S2_samp-2', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), st

MINDS SAMPLE FXS_KO_S2_samp-4
DICT hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-SPN.smr
FILE NAME hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-SPN.smr
FILE URL hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-2/hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-SPN.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-2/hbp-01853_cond-FXS_type-KO_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-SPN.smr
SUBJECT FXS_KO_S2
SAMPLE FXS_KO_S2_samp-5
ECoGExperiment(name='ECoG recording for FXS_KO_S2_samp-5', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), stimul

MINDS SAMPLE FXS_KO_S4_samp-1
DICT hbp-01853_cond-FXS_type-KO_sub-4_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
FILE NAME hbp-01853_cond-FXS_type-KO_sub-4_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
FILE URL hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-4/hbp-01853_cond-FXS_type-KO_sub-4_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/KO/cond-FXS_type-KO_sub-4/hbp-01853_cond-FXS_type-KO_sub-4_chs-16_area-S1_hem-RH_ana-ISO080_stim-EVK2.smr
SUBJECT FXS_KO_S4
SAMPLE FXS_KO_S4_samp-2
ECoGExperiment(name='ECoG recording for FXS_KO_S4_samp-2', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), st

MINDS SAMPLE FXS_WT_S1_samp-2
DICT hbp-01853_cond-FXS_type-WT_sub-1_chs-16_area-V1_hem-RH_ana-ISO100_stim-SPN.smr
FILE NAME hbp-01853_cond-FXS_type-WT_sub-1_chs-16_area-V1_hem-RH_ana-ISO100_stim-SPN.smr
FILE URL hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-1/hbp-01853_cond-FXS_type-WT_sub-1_chs-16_area-V1_hem-RH_ana-ISO100_stim-SPN.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-1/hbp-01853_cond-FXS_type-WT_sub-1_chs-16_area-V1_hem-RH_ana-ISO100_stim-SPN.smr
Subject(name='FXS_WT_S2', species=Species('Mus musculus', 'http://purl.obolibrary.org/obo/NCBITaxon_10090'), strain=Strain('C57BL/6', 'https://dknet.org/data/record/nlx_154697-1/IMSR_JAX:000664/resolver?q=%2A&l=%2A&filter[]=Catalog%20Number:000664&i=5d4227790fbc3b6016f250e8'), sex=Sex('male', 'schema:Male'), age=Age(QuantitativeValue(3 'months'), 'Post-natal'), id=None)
SUBJECT FXS_WT_S2
SAMPLE FXS_WT_S2_samp-1
ECoGExperiment(name

MINDS SAMPLE FXS_WT_S2_samp-3
DICT hbp-01853_cond-FXS_type-WT_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
FILE NAME hbp-01853_cond-FXS_type-WT_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
FILE URL hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-2/hbp-01853_cond-FXS_type-WT_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-2/hbp-01853_cond-FXS_type-WT_sub-2_chs-16_area-V1_hem-RH_ana-ISO080_stim-EVK1.smr
SUBJECT FXS_WT_S2
SAMPLE FXS_WT_S2_samp-4
ECoGExperiment(name='ECoG recording for FXS_WT_S2_samp-4', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), st

MINDS SAMPLE FXS_WT_S3_samp-2
DICT hbp-01853_cond-FXS_type-WT_sub-3_chs-16_area-S1_hem-RH_ana-ISO080_stim-SPN.smr
FILE NAME hbp-01853_cond-FXS_type-WT_sub-3_chs-16_area-S1_hem-RH_ana-ISO080_stim-SPN.smr
FILE URL hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-3/hbp-01853_cond-FXS_type-WT_sub-3_chs-16_area-S1_hem-RH_ana-ISO080_stim-SPN.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-3/hbp-01853_cond-FXS_type-WT_sub-3_chs-16_area-S1_hem-RH_ana-ISO080_stim-SPN.smr
SUBJECT FXS_WT_S3
SAMPLE FXS_WT_S3_samp-3
ECoGExperiment(name='ECoG recording for FXS_WT_S3_samp-3', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), stimul

MINDS SAMPLE FXS_WT_S3_samp-5
DICT hbp-01853_cond-FXS_type-WT_sub-3_chs-32_area-ALL_hem-RH_ana-ISO080_stim-SPN.smr
FILE NAME hbp-01853_cond-FXS_type-WT_sub-3_chs-32_area-ALL_hem-RH_ana-ISO080_stim-SPN.smr
FILE URL hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-3/hbp-01853_cond-FXS_type-WT_sub-3_chs-32_area-ALL_hem-RH_ana-ISO080_stim-SPN.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-3/hbp-01853_cond-FXS_type-WT_sub-3_chs-32_area-ALL_hem-RH_ana-ISO080_stim-SPN.smr
Subject(name='FXS_WT_S4', species=Species('Mus musculus', 'http://purl.obolibrary.org/obo/NCBITaxon_10090'), strain=Strain('C57BL/6', 'https://dknet.org/data/record/nlx_154697-1/IMSR_JAX:000664/resolver?q=%2A&l=%2A&filter[]=Catalog%20Number:000664&i=5d4227790fbc3b6016f250e8'), sex=Sex('male', 'schema:Male'), age=Age(QuantitativeValue(3 'months'), 'Post-natal'), id=None)
SUBJECT FXS_WT_S4
SAMPLE FXS_WT_S4_samp-1
ECoGExperiment(

MINDS SAMPLE FXS_WT_S4_samp-3
DICT hbp-01853_cond-FXS_type-WT_sub-4_chs-16_area-V1_hem-RH_ana-ISO090_stim-EVK1.smr
FILE NAME hbp-01853_cond-FXS_type-WT_sub-4_chs-16_area-V1_hem-RH_ana-ISO090_stim-EVK1.smr
FILE URL hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-4/hbp-01853_cond-FXS_type-WT_sub-4_chs-16_area-V1_hem-RH_ana-ISO090_stim-EVK1.smr
FILE LOCATION https://object.cscs.ch/v1/AUTH_25b4e28a742d4987a7b6f84c0c36512e/hbp-01853_FXS/data-originals/FXS/WT/cond-FXS_type-WT_sub-4/hbp-01853_cond-FXS_type-WT_sub-4_chs-16_area-V1_hem-RH_ana-ISO090_stim-EVK1.smr
SUBJECT FXS_WT_S4
SAMPLE FXS_WT_S4_samp-4
ECoGExperiment(name='ECoG recording for FXS_WT_S4_samp-4', device=Device(name='E16–100-S1-L6', manufacturer='Atlas Neuroengineering', model_name='E16-100-S1-L6', distribution=Distribution('http://www.atlasneuro.com/en/products/16-channel-probes/1032'), id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/electrophysiology/device/v0.1.0/81e9d453-5be6-493b-a8db-b6819707856d), st

In [18]:
trace_dict

{'FXS_KO_S1_samp-1': {'channel_names': ['E1',
   'E2',
   'E3',
   'E4',
   'E5',
   'E6',
   'E7',
   'E8',
   'E9',
   'E10',
   'E11',
   'E12',
   'E13',
   'E14',
   'E15',
   'E16',
   'HR'],
  'data_unit': ['V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V'],
  'time_step': QuantitativeValue(0.2016 'ms'),
  'sweeps': [0]},
 'FXS_KO_S1_samp-2': {'channel_names': ['E1',
   'E2',
   'E3',
   'E4',
   'E5',
   'E6',
   'E7',
   'E8',
   'E9',
   'E10',
   'E11',
   'E12',
   'E13',
   'E14',
   'E15',
   'E16',
   'HR'],
  'data_unit': ['V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V',
   'V'],
  'time_step': QuantitativeValue(0.2016 'ms'),
  'sweeps': [0]},
 'FXS_KO_S1_samp-3': {'channel_names': ['E1',
   'E2',
   'E3',
   'E4',
   'E5',
   'E6',
   'E7',
   'E8',
   'E9',
   'E10',
   'E11',
   'E12',
   'E13',
   'E14',
   'E1

In [19]:
with open(f'{minds_dataset.name}_trace_dict.yaml', 'w') as file: #save trace dict
    documents = yaml.dump(trace_dict, file)

In [20]:
print(f"Updated on: {datetime.now(tz=None)} on {client.nexus_endpoint}")

Updated on: 2020-06-22 14:47:35.638954 on https://nexus.humanbrainproject.org/v0
